In [35]:
from Teanaps import configure as con
PLOTLY_USERNAME = con.PLOTLY_USERNAME
PLOTLY_API_KEY = con.PLOTLY_API_KEY

import plotly 
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly import tools
plotly.tools.set_credentials_file(username=PLOTLY_USERNAME, api_key=PLOTLY_API_KEY)
plotly.tools.set_config_file(world_readable=False, sharing='private')
init_notebook_mode(connected=True)

from Teanaps.NaturalLanguageProcessing.SyntaxAnalyzer import SyntaxAnalyzer
from Teanaps.TextAnalytics.TfidfCalculator import TfidfCalculator
from Teanaps.TextAnalytics.TopicClustering import TopicClustering

#import statsmodels.api as sm
#from statsmodels.tsa.api import VAR, DynamicVAR
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
import numpy as np
import glob

In [4]:
df = pd.read_csv('kospi.txt', sep="\t") 
df["y"] = [float(y.replace(",", "")) for y in df["y"]]
df

,ds,y,오픈,고가,저가,거래량,변동폭
0,2019년 5월,2061.25,"2,192.94","2,217.03","2,051.69",-,-6.46%
1,2019년 4월,2203.59,"2,153.31","2,252.05","2,153.31",9.69B,2.94%
2,2019년 3월,2140.67,"2,210.97","2,216.02","2,120.93",6.46B,-2.49%
3,2019년 2월,2195.44,"2,211.93","2,241.76","2,167.36",6.88B,-0.43%
4,2019년 1월,2204.85,"2,050.55","2,222.88","1,984.53",9.46B,8.03%
5,2018년 12월,2041.04,"2,127.78","2,136.64","2,014.28",7.63B,-2.66%
6,2018년 11월,2096.86,"2,035.11","2,136.74","2,023.43",6.98B,3.31%
7,2018년 10월,2029.69,"2,349.64","2,352.11","1,985.95",7.02B,-13.37%
8,2018년 9월,2343.07,"2,317.93","2,356.62","2,264.90",5.43B,0.87%
9,2018년 8월,2322.88,"2,301.17","2,322.88","2,218.09",6.20B,1.20%


In [16]:
sa = SyntaxAnalyzer()

document_list = []
tokenized_sentence_list = []

POS_LIST = ["NNG", "NNP"]

num_monthly_documents = 100

file_list = glob.glob("monthly_data/*")
file_list.sort()
for filename in file_list:
    count = 1
    line_list = open(filename, encoding="utf-8").read().split("\n")
    for line in line_list:
        if count > num_monthly_documents:
            continue
        if len(line) > 100:
            count += 1
            col = line.split("\t")
            datetime = col[0]
            month = datetime[:7]
            content = col[1]
            document_list.append(content)
            tagged_word_list = sa.parse(content)
            tokenized_sentence = ""
            for word, pos in tagged_word_list:
                if pos in POS_LIST:
                    tokenized_sentence += " " + word
            tokenized_sentence_list.append(tokenized_sentence.strip())

In [17]:
tokenized_sentence_list

['귀주 가스 석탄 사용 금지 가격 급등 관련 상장 주식 대부분 연상 경향',
 '안녕 중국 거주 얼마 전 중국 주식 시장 투자 회사 투자 생각 글 귀주 가스 얼마 후 매입 갑자 장 정지 일전 프로 수익 매도 이후 한지 얼마 화 능수 전 투자 한지 얼 회사 투자 중국 주식 모두 성',
 '언어 매너 활동 알찬 해복 주위 가상 화폐 투자 기본 이득 지인 전 윌 투자 결과 현재 속 뉴스 한 코스닥 상승 열풍 주식 시장 폐장 코스닥 상장 시가총액 실체 리플 직 더리 움 가상 화폐 시가 총액 대한민국 미래 기업 전체 시가총액 가치 고수 고견 한 행복 정보 공유 비트 무료 리딩 매너 회원 신고 투자 본인 책임 정보 참고 판단 매너 반말 비속어 공격 어휘 방 홍보 시 경고 영구 강퇴 비트코인 비트 동호회 비트코인 채굴 라이트 코인 리플 가상 화폐 투자 정보 비트코인 시세 퀀텀 오스 비트코인 전망 비트 캐쉬 캐시 대쉬 클래식 빗 썸 업 비트 코 인원 코인 네스트',
 '존 버 기회 개인 욕심 급등 코인 손절 반복 결국 욕심 해 성 투하 쇼',
 '투자 코인 방법 걱정 스타일 폰',
 '암호 화폐 시장 한국 코스닥 정도 곳 비교 불가 앞 명함 블록체인 기술 자체 기업 정도 세계 인류 시장',
 '주식 판 리그 투자 가치 코인 초기 단계 접근 뻘 짓 대부분 수익 코인',
 '코스닥 시장 암호 화폐 거대 실체 표현 암호 화폐 이해',
 '파워 렛 매수 물 퀀텀 오미 세고 수익 후 매도 후 블록 티스 구매 후 지금 존 버',
 '인구 비밀 어머니 말 적극 공감 유사 문장 머릿속 후 예측 유일 지표 인구',
 '원래 투자 보고서 생각 대한항공 주식 시작 투자 경력 실패 좌절 경험 귀감 투자 기법 가투소 학생 입장 작성 남산 주성 은둔 고수 가투소 초기 선배 글 철학 방법론 학생 투자 소화 보고서 때 부끄러움 사실 의무감 보고서 댓글 가투소 활동 놀이터 역할 기록 의무감 올해 매수 매도 비용 회전율 의미 자체 투자 활동 시간 동안 가투소 활력 인사이트 다음 초심자 기록 예의 의무 생

In [18]:
tfidf = TfidfCalculator()

df_tfidf_monthly = []
for i in range(11):
    tokenized_sentence_list_monthly = tokenized_sentence_list[i*100:(i+1)*100]
    #tokenized_sentence_list_monthly
    tfidf.calculation_tfidf(tokenized_sentence_list_monthly)
    df_tfidf = tfidf.get_tfidf_matrix()
    #df_tfidf
    df_tfidf_monthly.append(df_tfidf)

In [25]:
df_tfidf_monthly[10]

,com,company,core,coredictionary,dictionary,fod,how,koreans,market,rice,...,후퇴,후회,휴식,휴켐스,흐름,흑삼,흑자,흡수,흥국,희망
0,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.080393,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
5,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.048898,0.0,0.000000,0.000000,0.000000,0.0
6,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
7,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
8,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
9,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0


In [26]:
tc = TopicClustering()

NUM_TOPICS = 10
NUM_KEYWORDS = 10

lda_result = tc.lda_topic_modeling(tokenized_sentence_list, NUM_TOPICS, NUM_KEYWORDS)
lda_result

[(0,
  [('코인', 0.021745967),
   ('비트코인', 0.017644873),
   ('화폐', 0.013680708),
   ('비트', 0.011545164),
   ('투자', 0.011124893),
   ('가상', 0.008696587),
   ('가격', 0.006786458),
   ('시세', 0.0062890085),
   ('리플', 0.0052589076),
   ('거래', 0.005136373)]),
 (1,
  [('미국', 0.013795516),
   ('중국', 0.013285835),
   ('무역', 0.012665658),
   ('경제', 0.011547268),
   ('美', 0.008615364),
   ('지수', 0.008003196),
   ('증시', 0.007680938),
   ('상승', 0.0073504383),
   ('미', 0.007107986),
   ('금리', 0.00639776)]),
 (2,
  [('투자', 0.019339904),
   ('매수', 0.015027211),
   ('수익', 0.012706295),
   ('주가', 0.011150643),
   ('공부', 0.007406968),
   ('매도', 0.0074069034),
   ('상승', 0.0068873535),
   ('차트', 0.0062085055),
   ('급등', 0.005357988),
   ('회원', 0.004651145)]),
 (3,
  [('보통주', 0.026981765),
   ('투자', 0.015421236),
   ('기업', 0.010486162),
   ('장주식', 0.007987481),
   ('비상', 0.007971582),
   ('거래', 0.006609883),
   ('주가', 0.0061952965),
   ('장외', 0.0058277757),
   ('가치', 0.0051311506),
   ('방법', 0.0048889387)]),
 

In [29]:
hot_keyword_list = []

for num_topics, keyword_list in lda_result:
    for keyword, score in keyword_list:
        if keyword not in hot_keyword_list:
            hot_keyword_list.append(keyword)
            
hot_keyword_dict = {}

for hot_keyword in hot_keyword_list:
    tfidf_series = []
    for df_tfidf in df_tfidf_monthly:
        tfidf_dict = dict(df_tfidf.sum())
        if hot_keyword in tfidf_dict.keys():
            tfidf_series.append(tfidf_dict[hot_keyword])
        else:
            tfidf_series.append(0)
    tfidf_series2 = []
    for idx in range(len(tfidf_series)-1):
        if tfidf_series[idx] > tfidf_series[idx+1]:
            tfidf_series2.append(0)
        else:
            tfidf_series2.append(1)
    hot_keyword_dict[hot_keyword] = tfidf_series2

{'美': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 '가격': [1, 0, 1, 0, 1, 0, 1, 1, 0, 1],
 '가상': [0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
 '가치': [1, 0, 1, 1, 0, 0, 0, 1, 1, 1],
 '개인': [1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 '거래': [0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
 '경제': [0, 0, 1, 1, 0, 1, 1, 0, 0, 0],
 '공급': [1, 0, 1, 0, 1, 0, 1, 1, 0, 0],
 '공부': [1, 1, 0, 1, 0, 1, 0, 0, 0, 0],
 '금리': [0, 0, 1, 0, 0, 1, 1, 0, 0, 1],
 '금융': [0, 0, 1, 0, 0, 1, 1, 0, 1, 1],
 '급등': [1, 0, 1, 0, 1, 0, 1, 1, 1, 0],
 '기간': [1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
 '기관': [1, 0, 1, 1, 1, 0, 1, 0, 1, 0],
 '기업': [0, 0, 1, 1, 0, 1, 0, 0, 1, 1],
 '대출': [0, 0, 1, 0, 1, 0, 1, 0, 0, 1],
 '돈': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 '리플': [0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
 '매도': [0, 0, 0, 1, 0, 1, 1, 0, 1, 0],
 '매수': [1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
 '모단': [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 '목표': [1, 0, 1, 0, 0, 1, 0, 1, 0, 0],
 '무역': [0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
 '미': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 '미국': [0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
 '박스': [1, 0, 1, 1, 0, 1, 0,

In [30]:
d = df[["y"]].values[6:17]
d = list(d)
d = [float(dd) for dd in d]
d

[2096.86,
 2029.69,
 2343.07,
 2322.88,
 2295.26,
 2326.13,
 2423.01,
 2515.38,
 2445.85,
 2427.36,
 2566.46]

In [31]:
x = [i for i in range(11)]

data = [go.Scatter(
        x=x,
        y=d,
        name="KOSPI",
        marker=dict(
            #color='blue',
        ),
        yaxis='y2'
    )]


for key, value in hot_keyword_dict.items():
    y = value
    data.append(go.Scatter(
        x=x,
        y=y,
        name=key,
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    ),)
layout = go.Layout(
    title='Sequence LDA Model Topic Trend',
    xaxis=dict(
        title='TIME SLICE',
        titlefont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='WEIGHT',
        titlefont=dict(
            size=10,
            color='black'
        ),
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all',
        #overlaying='y',
    ),
    yaxis2=dict(
                title='TF-IDF',
                titlefont=dict(
                    size=10,
                    color='black'
                ),
                showticklabels=True,
                tickangle=0,
                tickfont=dict(
                    size=10,
                    color='black'
                ),
                overlaying='y',
                side='right'
            ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='TF-IDF Graph')

In [33]:
y = df["y"].values[6:17]
ud_y = []

for y_idx in range(len(y)-1):
    print(y[y_idx], y[y_idx+1])
    if y[y_idx] < y[y_idx+1]:
        ud_y.append(1)
    else:
        ud_y.append(0)

2096.86 2029.69
2029.69 2343.07
2343.07 2322.88
2322.88 2295.26
2295.26 2326.13
2326.13 2423.01
2423.01 2515.38
2515.38 2445.85
2445.85 2427.36
2427.36 2566.46


In [34]:
Y = pd.DataFrame(ud_y)
Y = Y[0].values

X = pd.DataFrame(hot_keyword_dict)
X = X.values[:]

df = pd.DataFrame(X, columns = hot_keyword_dict.keys())
df['target'] = Y
df

,코인,비트코인,화폐,비트,투자,가상,가격,시세,리플,거래,미국,중국,무역,경제,美,지수,증시,상승,미,금리,매수,수익,주가,공부,매도,차트,급등,회원,보통주,기업,장주식,비상,장외,가치,방법,금융,대출,하락,세계,은행,부동산,모단,삼성,공급,재무,추천,인구,펀드,증권,손실,정부,사람,돈,기관,서울,세금,시작,분석,서비스,추세,진입,개인,상장,박스,기간,목표,target
0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,0,1,1,1,1,0,1,0,1,1,0,0,0,1,0,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,1,0,1,1,1,1,0,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1
2,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0
3,1,0,0,0,1,0,0,1,1,0,1,0,0,1,1,0,0,0,1,0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,1,0,1,0,0,1,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0
4,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,0,1,0,1,1,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1
5,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,1,0,1,0,1,1,0,1,1,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1
6,0,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,0,0,1,1,1,0,1,1,1,0,1,0,1,1,0,1,0,0,1,1,1,1,1,0,0,1,1,0,0,0,0,1,0,1
7,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,1,0,1,0,1,1,0,0,0,0,0,0,1,1,0,1,0
8,1,1,1,1,1,0,0,1,1,1,0,1,0,0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,0,1,0,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,0
9,1,1,1,1,0,1,1,0,1,0,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1,1,1,1,1,0,1,1,0,1,0,0,1,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1


In [83]:
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=None)
    dtc = DecisionTreeClassifier(random_state=0)
    #dtc = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
    dtc.fit(x_train, y_train)
    score = dtc.score(x_test, y_test)
    print('Accuracy', i+1, ' :', score)

Accuracy 1  : 0.6666666666666666
Accuracy 2  : 0.6666666666666666
Accuracy 3  : 0.6666666666666666
Accuracy 4  : 0.6666666666666666
Accuracy 5  : 0.3333333333333333
Accuracy 6  : 0.3333333333333333
Accuracy 7  : 0.6666666666666666
Accuracy 8  : 0.6666666666666666
Accuracy 9  : 0.6666666666666666
Accuracy 10  : 0.6666666666666666


In [84]:
x = [i for i in range(11)]

data = [go.Scatter(
        x=x,
        y=d,
        name="KOSPI",
        marker=dict(
            #color='blue',
        ),
        yaxis='y2'
    ),
    go.Scatter(
        x=x,
        y=ud_y,
        name="Label",
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    ),
    go.Scatter(
        x=x,
        y=dtc.predict(X),
        name="Predict",
        marker=dict(
            #color='blue',
        ),
        yaxis='y'
    )]

layout = go.Layout(
    title='Predictive Analysis',
    xaxis=dict(
        title='TIME SLICE',
        titlefont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='DOWN <-------------------------------------------> UP',
        titlefont=dict(
            size=10,
            color='black'
        ),
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            size=10,
            color='black'
        ),
        dtick = 1,
        exponentformat='e',
        showexponent='all',
        #overlaying='y',
    ),
    yaxis2=dict(
                title='KOSPI200',
                titlefont=dict(
                    size=10,
                    color='black'
                ),
                showticklabels=True,
                tickangle=0,
                tickfont=dict(
                    size=10,
                    color='black'
                ),
                overlaying='y',
                side='right'
            ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='TF-IDF Graph')